# Professional DIY Guide Generator

Generate comprehensive, professional-quality home improvement guides using Google's Gemini AI.

**Features:**
- Complete step-by-step instructions with materials, tools, and safety tips
- Optional AI-generated images for each step
- Clean, readable markdown output
- Customizable for any DIY topic

**Reference Style:** Similar to professional retail guides (e.g., Home Depot), but with 100% original content.

## 📋 Prerequisites

**Required:**
- Google AI Studio API key ([Get one here](https://aistudio.google.com/app/apikey))
- Python packages: `google-genai`, `pillow`

**Setup Instructions:**

1. Set your API key as an environment variable:
   ```powershell
   $env:GOOGLE_API_KEY = "your-api-key-here"
   ```

2. Install required packages (uncomment the cell below if needed)

In [ ]:
# Uncomment to install required packages
# %pip install --quiet google-genai pillow

## 🔧 Configuration & Setup

In [ ]:
import os
import json
import re
import textwrap
import pathlib
import base64
import io
from dataclasses import dataclass, field
from typing import List, Optional, Any

import google.generativeai as genai
from google import genai as genai_client
from google.genai import types
from PIL import Image
from IPython.display import display, Markdown, HTML

# Verify API key is set
API_KEY = os.getenv('GOOGLE_API_KEY')
if not API_KEY:
    raise RuntimeError(
        '❌ Missing GOOGLE_API_KEY environment variable.\n'
        'Please set it and restart the kernel.\n'
        'PowerShell: $env:GOOGLE_API_KEY = "your-key"'
    )

# Configure Gemini for text generation
genai.configure(api_key=API_KEY)

# Initialize Imagen client for image generation
imagen_client = genai_client.Client(api_key=API_KEY)

# Model configuration
TEXT_MODEL_NAME = 'gemini-2.5-flash'  # Use 'gemini-1.5-flash' for faster/cheaper generation
IMAGE_MODEL_NAME = 'imagen-4.0-generate-001'  # Imagen 3 for image generation

print('✅ Configuration loaded successfully!')
print(f'📝 Text Model: {TEXT_MODEL_NAME}')
print(f'🎨 Image Model: {IMAGE_MODEL_NAME}')

## 📚 Data Structures

In [ ]:
@dataclass
class GuideStep:
    """Represents a single step in the DIY guide."""
    title: str
    body_md: str
    tips: Optional[List[str]] = None

@dataclass
class DIYGuide:
    """Complete DIY guide structure."""
    topic: str
    difficulty: str
    duration_estimate: str
    overview_md: str
    materials: List[str]
    tools: List[str]
    safety_tips: List[str]
    steps: List[GuideStep]
    cleanup_disposal: Optional[str] = None
    maintenance_tips: Optional[str] = None
    image_prompts: Optional[List[str]] = None

print('✅ Data structures defined')

## 🤖 Initialize AI Models

In [ ]:
# Initialize text generation model
text_model = genai.GenerativeModel(TEXT_MODEL_NAME)
print(f'✅ Text model initialized: {TEXT_MODEL_NAME}')

# Imagen client is already initialized in the configuration cell
print(f'✅ Image generation client ready: {IMAGE_MODEL_NAME}')

## 🎯 Guide Generation Functions

In [ ]:
def build_guide_prompt(topic: str, audience: str = 'homeowners', style: str = 'professional') -> str:
    """Build a comprehensive prompt for guide generation."""
    
    system_instructions = textwrap.dedent(f'''
    You are an expert DIY home improvement guide writer with 20+ years of experience.
    
    Create a comprehensive, {style}, and safety-focused guide that includes:
    - Clear overview explaining the project and benefits
    - Complete materials list with specific quantities/sizes where applicable
    - All required tools
    - Important safety precautions
    - Step-by-step instructions with detailed explanations
    - Pro tips for each step when relevant
    - Cleanup and disposal instructions
    - Maintenance recommendations
    
    Requirements:
    - Use US measurements (inches, feet, gallons, etc.)
    - Reference common US retail product names (Home Depot, Lowe's brands)
    - Include realistic time estimates
    - Rate difficulty honestly (Beginner/Intermediate/Advanced)
    - Note when permits or inspections may be required
    - Keep all content 100% original
    - Use clear, concise language suitable for {audience}
    ''')
    
    json_schema = textwrap.dedent('''
    Return ONLY valid JSON (no markdown formatting) with this exact structure:
    {
      "topic": "string",
      "difficulty": "Beginner|Intermediate|Advanced",
      "duration_estimate": "string (e.g., '4-6 hours', '1-2 days')",
      "overview_md": "string (markdown)",
      "materials": ["string"],
      "tools": ["string"],
      "safety_tips": ["string"],
      "steps": [
        {
          "title": "string",
          "body_md": "string (markdown)",
          "tips": ["string"] (optional)
        }
      ],
      "cleanup_disposal": "string (markdown, optional)",
      "maintenance_tips": "string (markdown, optional)"
    }
    ''')
    
    return f"{system_instructions}\n\nTopic: {topic}\n\n{json_schema}"


def generate_guide(topic: str, audience: str = 'homeowners', style: str = 'professional') -> DIYGuide:
    """Generate a complete DIY guide using Gemini."""
    
    print(f'🔄 Generating guide for: {topic}...')
    
    prompt = build_guide_prompt(topic, audience, style)
    response = text_model.generate_content(prompt)
    response_text = response.text if hasattr(response, 'text') else str(response)
    
    # Parse JSON from response
    try:
        # Remove markdown code blocks if present
        response_text = re.sub(r'^```json\s*', '', response_text)
        response_text = re.sub(r'\s*```$', '', response_text)
        data = json.loads(response_text)
    except json.JSONDecodeError:
        # Try to extract JSON from mixed content
        match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response_text, re.DOTALL)
        if not match:
            raise ValueError(f'Could not parse JSON from response.\n\nFirst 500 chars:\n{response_text[:500]}')
        data = json.loads(match.group(0))
    
    # Convert steps to GuideStep objects
    steps = [GuideStep(**step) for step in data.get('steps', [])]
    
    guide = DIYGuide(
        topic=data.get('topic', topic),
        difficulty=data.get('difficulty', 'Intermediate'),
        duration_estimate=data.get('duration_estimate', 'Varies'),
        overview_md=data.get('overview_md', ''),
        materials=data.get('materials', []),
        tools=data.get('tools', []),
        safety_tips=data.get('safety_tips', []),
        steps=steps,
        cleanup_disposal=data.get('cleanup_disposal'),
        maintenance_tips=data.get('maintenance_tips')
    )
    
    print(f'✅ Guide generated successfully! ({len(steps)} steps)')
    return guide


def render_guide_markdown(guide: DIYGuide) -> str:
    """Render DIYGuide as formatted markdown."""
    
    lines = []
    
    # Header
    lines.append(f'# {guide.topic}\n')
    lines.append(f'**Difficulty:** {guide.difficulty} | **Time:** {guide.duration_estimate}\n')
    lines.append('---\n')
    
    # Overview
    lines.append('## 📖 Overview\n')
    lines.append(f'{guide.overview_md}\n')
    
    # Materials
    if guide.materials:
        lines.append('## 🛒 Materials Needed\n')
        for material in guide.materials:
            lines.append(f'- {material}')
        lines.append('')
    
    # Tools
    if guide.tools:
        lines.append('## 🔧 Tools Required\n')
        for tool in guide.tools:
            lines.append(f'- {tool}')
        lines.append('')
    
    # Safety
    if guide.safety_tips:
        lines.append('## ⚠️ Safety Tips\n')
        for tip in guide.safety_tips:
            lines.append(f'- {tip}')
        lines.append('')
    
    # Steps
    lines.append('## 📝 Step-by-Step Instructions\n')
    for i, step in enumerate(guide.steps, 1):
        lines.append(f'### Step {i}: {step.title}\n')
        lines.append(f'{step.body_md}\n')
        
        if step.tips:
            lines.append('**💡 Pro Tips:**')
            for tip in step.tips:
                lines.append(f'- {tip}')
            lines.append('')
    
    # Cleanup
    if guide.cleanup_disposal:
        lines.append('## 🧹 Cleanup and Disposal\n')
        lines.append(f'{guide.cleanup_disposal}\n')
    
    # Maintenance
    if guide.maintenance_tips:
        lines.append('## 🔄 Ongoing Maintenance\n')
        lines.append(f'{guide.maintenance_tips}\n')
    
    return '\n'.join(lines)

print('✅ Guide generation functions ready')

## 🎨 Image Generation Functions

In [ ]:
def create_image_prompts(guide: DIYGuide, max_steps: int = 6) -> List[str]:
    """Generate image prompts for guide visualization."""
    
    prompts = []
    
    # Hero/overview image
    prompts.append(
        f"Professional photography of completed {guide.topic.lower()}, "
        f"bright natural lighting, high quality, photorealistic, 4K resolution, "
        f"clean composition, home improvement setting"
    )
    
    # Step-by-step images
    for i, step in enumerate(guide.steps[:max_steps], 1):
        prompts.append(
            f"Step {i} of {guide.topic.lower()}: {step.title}, "
            f"instructional diagram style, clear and detailed, photorealistic, "
            f"professional home improvement photography, well-lit, neutral background"
        )
    
    return prompts


def generate_images(
    prompts: List[str],
    output_dir: str = '../generated_images',
    prefix: str = 'diy',
    num_images_per_prompt: int = 1
) -> List[str]:
    """Generate and save images from prompts using Imagen API."""
    
    output_path = pathlib.Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    saved_paths = []
    
    print(f'\n🎨 Generating {len(prompts)} images...')
    
    for idx, prompt in enumerate(prompts, 1):
        try:
            print(f'  [{idx}/{len(prompts)}] Generating...', end=' ')
            
            # Use new Imagen API with genai.Client
            response = imagen_client.models.generate_images(
                model=IMAGE_MODEL_NAME,
                prompt=prompt,
                config=types.GenerateImagesConfig(
                    number_of_images=num_images_per_prompt,
                    safety_filter_level='block_low_and_above',  # Fixed: use supported safety level
                    person_generation='allow_adult'
                )
            )
            
            # Process generated images
            if response.generated_images:
                for img_idx, generated_image in enumerate(response.generated_images):
                    # Get PIL Image from response
                    img = generated_image.image
                    
                    # Save image
                    if num_images_per_prompt > 1:
                        file_path = output_path / f'{prefix}_{idx:02d}_{img_idx+1}.png'
                    else:
                        file_path = output_path / f'{prefix}_{idx:02d}.png'
                    
                    img.save(file_path)
                    saved_paths.append(str(file_path))
                
                print(f'✅ Saved to {file_path.name}')
            else:
                print('❌ No images generated')
        
        except Exception as e:
            print(f'❌ Error: {str(e)[:100]}')
    
    print(f'\n✅ Generated {len(saved_paths)}/{len(prompts)} images successfully\n')
    return saved_paths

print('✅ Image generation functions ready')

## 🚀 Generate Your DIY Guide

Edit the `TOPIC` variable below to generate guides for any home improvement project.

**Example topics:**
- How to Install a French Drain
- How to Replace a Bathroom Faucet
- How to Install Laminate Flooring
- How to Build a Deck
- How to Install a Ceiling Fan

In [ ]:
# Configuration
TOPIC = 'How to Install a French Drain'
AUDIENCE = 'homeowners'  # or 'beginners', 'professionals', etc.
STYLE = 'professional'   # or 'casual', 'technical', etc.

# Reference for style inspiration (not used for content generation)
REFERENCE_URL = 'https://www.homedepot.com/c/ah/how-to-install-a-french-drain/9ba683603be9fa5395fab9012cc2665'

print(f'\n🎯 Generating guide: {TOPIC}\n')
print(f'   Target audience: {AUDIENCE}')
print(f'   Writing style: {STYLE}')
print(f'   Reference style: {REFERENCE_URL}\n')

# Update model name to supported version
TEXT_MODEL_NAME = 'gemini-2.5-flash'  # Use supported model
text_model = genai.GenerativeModel(TEXT_MODEL_NAME)

# Generate the guide
guide = generate_guide(topic=TOPIC, audience=AUDIENCE, style=STYLE)

# Render and display
markdown_output = render_guide_markdown(guide)
display(Markdown(markdown_output))

## 🎨 Generate Images (Optional)

Generate professional images to accompany your guide. Images are saved to `../generated_images/`.

**Note:** Image generation requires Imagen access on your API key and may incur additional costs.

In [ ]:
# Generate image prompts
image_prompts = create_image_prompts(guide, max_steps=6)
guide.image_prompts = image_prompts

print('\n📋 Generated Image Prompts:\n')
for i, prompt in enumerate(image_prompts, 1):
    print(f'{i}. {prompt[:100]}...')

# Generate actual images (if model is available)
image_paths = generate_images(
    prompts=image_prompts,
    output_dir='../generated_images',
    prefix='french_drain'
)

# Display generated images in the notebook
if image_paths:
    print('\n📁 Saved images:')
    for path in image_paths:
        print(f'   {path}')
    
    print('\n🖼️ Displaying generated images:\n')
    for i, path in enumerate(image_paths, 1):
        try:
            img = Image.open(path)
            # Add title for each image
            if i == 1:
                display(HTML(f'<h3>Hero Image</h3>'))
            else:
                display(HTML(f'<h3>Step {i-1} Image</h3>'))
            # Display the image
            display(img)
        except Exception as e:
            print(f'Error displaying {path}: {e}')
else:
    print('\n⚠️ No images were generated. Check the errors above or verify Imagen API access.')

## 💾 Export Guide

Export your guide as a standalone markdown file.

In [ ]:
def export_guide(guide: DIYGuide, output_dir: str = '../exports') -> str:
    """Export guide as markdown file."""
    
    output_path = pathlib.Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Create safe filename
    safe_name = re.sub(r'[^a-zA-Z0-9\s-]', '', guide.topic)
    safe_name = re.sub(r'\s+', '_', safe_name).lower()
    file_path = output_path / f'{safe_name}.md'
    
    # Write markdown content
    markdown_content = render_guide_markdown(guide)
    file_path.write_text(markdown_content, encoding='utf-8')
    
    return str(file_path)

# Export the guide
export_path = export_guide(guide)
print(f'\n✅ Guide exported to: {export_path}')

In [ ]:

def search_products_with_grounding(materials: List[str], tools: List[str]) -> dict:
    """
    Use Google Search Grounding to find real products for materials and tools.
    
    Args:
        materials: List of materials needed
        tools: List of tools required
    
    Returns:
        Dictionary with product information and purchase links
    """
    product_results = {
        'materials': {},
        'tools': {}
    }
    
    print('\n🔍 Searching for products with Google Grounding...\n')
    
    # Configure grounding tool
    grounding_tool = types.Tool(google_search=types.GoogleSearch())
    config = types.GenerateContentConfig(
        tools=[grounding_tool],
        temperature=0.3,
        response_mime_type="application/json"
    )
    
    # Search for materials
    print('📦 Finding materials...')
    for i, material in enumerate(materials[:10], 1):  # Limit to avoid rate limits
        try:
            print(f'  [{i}/{min(len(materials), 10)}] Searching: {material[:50]}...', end=' ')
            
            prompt = f"""Find the best product to buy for this DIY material: {material}

Provide current pricing and availability information.

Return ONLY valid JSON with this structure:
{{
    "product_name": "specific product name",
    "brand": "brand name",
    "price_range": "$X - $Y",
    "retailers": ["Home Depot", "Lowe's", "Amazon"],
    "specifications": "key specs",
    "purchase_url": "direct link if available"
}}
"""
            
            response = imagen_client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt,
                config=config
            )
            
            # Parse response
            try:
                product_info = json.loads(response.text)
                product_results['materials'][material] = product_info
                print(f'✅ Found: {product_info.get("product_name", "Product")[:40]}')
            except json.JSONDecodeError:
                print('⚠️ Parsing error')
                product_results['materials'][material] = {
                    "product_name": material,
                    "note": "Search manually at Home Depot or Lowe's",
                    "retailers": ["Home Depot", "Lowe's"]
                }
            
        except Exception as e:
            print(f'❌ Error: {str(e)[:50]}')
            product_results['materials'][material] = {
                "product_name": material,
                "note": "Search manually"
            }
    
    # Search for tools
    print('\n🔧 Finding tools...')
    for i, tool in enumerate(tools[:10], 1):
        try:
            print(f'  [{i}/{min(len(tools), 10)}] Searching: {tool[:50]}...', end=' ')
            
            prompt = f"""Find the best tool product for: {tool}

Provide current pricing and availability information.

Return ONLY valid JSON with this structure:
{{
    "product_name": "specific product/model name",
    "brand": "brand name",
    "price_range": "$X - $Y",
    "retailers": ["Home Depot", "Lowe's", "Amazon"],
    "features": "key features",
    "purchase_url": "direct link if available"
}}
"""
            
            response = imagen_client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt,
                config=config
            )
            
            try:
                product_info = json.loads(response.text)
                product_results['tools'][tool] = product_info
                print(f'✅ Found: {product_info.get("product_name", "Product")[:40]}')
            except json.JSONDecodeError:
                print('⚠️ Parsing error')
                product_results['tools'][tool] = {
                    "product_name": tool,
                    "note": "Search manually at Home Depot or Lowe's",
                    "retailers": ["Home Depot", "Lowe's"]
                }
            
        except Exception as e:
            print(f'❌ Error: {str(e)[:50]}')
            product_results['tools'][tool] = {
                "product_name": tool,
                "note": "Search manually"
            }
    
    print(f'\n✅ Product search complete!\n')
    return product_results


def display_product_results(product_results: dict):
    """Display product search results in a formatted way."""
    
    print('\n' + '='*80)
    print('🛒 SHOPPING LIST WITH PURCHASE LINKS')
    print('='*80 + '\n')
    
    # Materials
    if product_results['materials']:
        print('📦 MATERIALS TO BUY:\n')
        for i, (material, info) in enumerate(product_results['materials'].items(), 1):
            print(f'{i}. {material}')
            print(f'   ➜ Product: {info.get("product_name", "N/A")}')
            if 'brand' in info:
                print(f'   ➜ Brand: {info["brand"]}')
            if 'price_range' in info:
                print(f'   ➜ Price: {info["price_range"]}')
            if 'retailers' in info:
                print(f'   ➜ Buy at: {", ".join(info["retailers"])}')
            if 'purchase_url' in info and info['purchase_url']:
                print(f'   ➜ Link: {info["purchase_url"]}')
            if 'specifications' in info:
                print(f'   ➜ Specs: {info["specifications"]}')
            if 'note' in info:
                print(f'   ➜ Note: {info["note"]}')
            print()
    
    # Tools
    if product_results['tools']:
        print('\n🔧 TOOLS TO BUY:\n')
        for i, (tool, info) in enumerate(product_results['tools'].items(), 1):
            print(f'{i}. {tool}')
            print(f'   ➜ Product: {info.get("product_name", "N/A")}')
            if 'brand' in info:
                print(f'   ➜ Brand: {info["brand"]}')
            if 'price_range' in info:
                print(f'   ➜ Price: {info["price_range"]}')
            if 'retailers' in info:
                print(f'   ➜ Buy at: {", ".join(info["retailers"])}')
            if 'purchase_url' in info and info['purchase_url']:
                print(f'   ➜ Link: {info["purchase_url"]}')
            if 'features' in info:
                print(f'   ➜ Features: {info["features"]}')
            if 'note' in info:
                print(f'   ➜ Note: {info["note"]}')
            print()
    
    print('='*80 + '\n')

print('✅ Product search functions ready')

In [ ]:

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, PageBreak, ListFlowable, ListItem
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from reportlab.lib.colors import HexColor

def export_guide_to_pdf(
    guide: DIYGuide, 
    image_paths: List[str] = None,
    output_dir: str = '../exports'
) -> str:
    """
    Export DIY guide with images to a professional PDF document.
    
    Args:
        guide: The DIYGuide object to export
        image_paths: List of image file paths (hero image first, then step images)
        output_dir: Directory to save the PDF
    
    Returns:
        Path to the generated PDF file
    """
    output_path = pathlib.Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Create safe filename
    safe_name = re.sub(r'[^a-zA-Z0-9\s-]', '', guide.topic)
    safe_name = re.sub(r'\s+', '_', safe_name).lower()
    pdf_path = output_path / f'{safe_name}.pdf'
    
    # Create PDF document
    doc = SimpleDocTemplate(
        str(pdf_path),
        pagesize=letter,
        rightMargin=0.75*inch,
        leftMargin=0.75*inch,
        topMargin=0.75*inch,
        bottomMargin=0.75*inch
    )
    
    # Define styles
    styles = getSampleStyleSheet()
    
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=HexColor('#1a1a1a'),
        spaceAfter=12,
        alignment=TA_CENTER,
        fontName='Helvetica-Bold'
    )
    
    subtitle_style = ParagraphStyle(
        'CustomSubtitle',
        parent=styles['Normal'],
        fontSize=11,
        textColor=HexColor('#666666'),
        spaceAfter=20,
        alignment=TA_CENTER,
        fontName='Helvetica'
    )
    
    heading2_style = ParagraphStyle(
        'CustomHeading2',
        parent=styles['Heading2'],
        fontSize=16,
        textColor=HexColor('#2c5aa0'),
        spaceAfter=10,
        spaceBefore=15,
        fontName='Helvetica-Bold'
    )
    
    heading3_style = ParagraphStyle(
        'CustomHeading3',
        parent=styles['Heading3'],
        fontSize=13,
        textColor=HexColor('#1a1a1a'),
        spaceAfter=8,
        spaceBefore=12,
        fontName='Helvetica-Bold'
    )
    
    body_style = ParagraphStyle(
        'CustomBody',
        parent=styles['Normal'],
        fontSize=10,
        textColor=HexColor('#333333'),
        spaceAfter=10,
        leading=14,
        fontName='Helvetica'
    )
    
    tip_style = ParagraphStyle(
        'TipStyle',
        parent=styles['Normal'],
        fontSize=9,
        textColor=HexColor('#0066cc'),
        spaceAfter=6,
        leftIndent=20,
        fontName='Helvetica-Oblique'
    )
    
    # Build PDF content
    story = []
    
    # Title
    story.append(Paragraph(guide.topic, title_style))
    story.append(Spacer(1, 0.1*inch))
    
    # Subtitle with difficulty and time
    subtitle_text = f"<b>Difficulty:</b> {guide.difficulty} | <b>Time:</b> {guide.duration_estimate}"
    story.append(Paragraph(subtitle_text, subtitle_style))
    
    # Hero image (if available)
    if image_paths and len(image_paths) > 0:
        try:
            img = RLImage(image_paths[0], width=5*inch, height=3.5*inch)
            story.append(img)
            story.append(Spacer(1, 0.2*inch))
        except Exception as e:
            print(f"Warning: Could not add hero image: {e}")
    
    story.append(Spacer(1, 0.1*inch))
    
    # Overview
    story.append(Paragraph("📖 Overview", heading2_style))
    overview_paras = guide.overview_md.split('\n\n')
    for para in overview_paras:
        if para.strip():
            story.append(Paragraph(para.strip(), body_style))
    story.append(Spacer(1, 0.15*inch))
    
    # Materials
    if guide.materials:
        story.append(Paragraph("🛒 Materials Needed", heading2_style))
        material_items = [ListItem(Paragraph(mat, body_style)) for mat in guide.materials]
        story.append(ListFlowable(material_items, bulletType='bullet'))
        story.append(Spacer(1, 0.15*inch))
    
    # Tools
    if guide.tools:
        story.append(Paragraph("🔧 Tools Required", heading2_style))
        tool_items = [ListItem(Paragraph(tool, body_style)) for tool in guide.tools]
        story.append(ListFlowable(tool_items, bulletType='bullet'))
        story.append(Spacer(1, 0.15*inch))
    
    # Safety Tips
    if guide.safety_tips:
        story.append(Paragraph("⚠️ Safety Tips", heading2_style))
        safety_items = [ListItem(Paragraph(tip, body_style)) for tip in guide.safety_tips]
        story.append(ListFlowable(safety_items, bulletType='bullet'))
        story.append(Spacer(1, 0.2*inch))
    
    # Page break before steps
    story.append(PageBreak())
    
    # Step-by-Step Instructions
    story.append(Paragraph("📝 Step-by-Step Instructions", heading2_style))
    story.append(Spacer(1, 0.1*inch))
    
    for i, step in enumerate(guide.steps, 1):
        # Step image (if available) - hero image is at index 0, so step images start at index 1
        if image_paths and len(image_paths) > i:
            try:
                img = RLImage(image_paths[i], width=4*inch, height=2.8*inch)
                story.append(img)
                story.append(Spacer(1, 0.1*inch))
            except Exception as e:
                print(f"Warning: Could not add image for step {i}: {e}")
        
        # Step title
        story.append(Paragraph(f"Step {i}: {step.title}", heading3_style))
        
        # Step body
        body_paras = step.body_md.split('\n\n')
        for para in body_paras:
            if para.strip():
                story.append(Paragraph(para.strip(), body_style))
        
        # Pro tips
        if step.tips:
            story.append(Spacer(1, 0.05*inch))
            story.append(Paragraph("💡 <b>Pro Tips:</b>", body_style))
            for tip in step.tips:
                story.append(Paragraph(f"• {tip}", tip_style))
        
        story.append(Spacer(1, 0.2*inch))
    
    # Cleanup
    if guide.cleanup_disposal:
        story.append(PageBreak())
        story.append(Paragraph("🧹 Cleanup and Disposal", heading2_style))
        cleanup_paras = guide.cleanup_disposal.split('\n\n')
        for para in cleanup_paras:
            if para.strip():
                story.append(Paragraph(para.strip(), body_style))
        story.append(Spacer(1, 0.2*inch))
    
    # Maintenance
    if guide.maintenance_tips:
        story.append(Paragraph("🔄 Ongoing Maintenance", heading2_style))
        maintenance_paras = guide.maintenance_tips.split('\n\n')
        for para in maintenance_paras:
            if para.strip():
                story.append(Paragraph(para.strip(), body_style))
    
    # Build PDF
    doc.build(story)
    
    return str(pdf_path)

print('✅ PDF export function ready')


# Export guide to PDF with images
pdf_path = export_guide_to_pdf(guide, image_paths=image_paths)

print(f'\n📄 PDF Export Complete!')
print(f'📁 Location: {pdf_path}')
print(f'\n📊 PDF Contents:')
print(f'   ✅ Title and metadata')
print(f'   ✅ Hero image')
print(f'   ✅ Overview section')
print(f'   ✅ Materials list ({len(guide.materials)} items)')
print(f'   ✅ Tools list ({len(guide.tools)} items)')
print(f'   ✅ Safety tips ({len(guide.safety_tips)} items)')
print(f'   ✅ {len(guide.steps)} step-by-step instructions with images')
if guide.cleanup_disposal:
    print(f'   ✅ Cleanup and disposal')
if guide.maintenance_tips:
    print(f'   ✅ Maintenance recommendations')

## 📝 Usage Tips

**Customization:**
- Change `TOPIC` to any DIY project
- Adjust `AUDIENCE` for different skill levels
- Modify `STYLE` for different writing tones
- Edit `TEXT_MODEL_NAME` to switch models:
  - `gemini-1.5-pro` - Best quality, slower
  - `gemini-1.5-flash` - Faster, cheaper, good quality

**Improving Results:**
- Be specific in your topic (e.g., "How to Install a French Drain in Clay Soil")
- Modify system instructions in `build_guide_prompt()` for specialized needs
- Run generation multiple times and pick the best result

**Image Generation:**
- Requires Imagen API access
- Edit prompts in `create_image_prompts()` for different styles
- Images are saved as PNG files with sequential numbering

**Troubleshooting:**
- If JSON parsing fails, the model may have added extra text - check the error message
- Image generation errors are non-fatal; guide will still work without images
- API rate limits may require waiting between requests